In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"
%matplotlib inline

/Users/csuarezgurruchaga/Desktop/Digital-House/clase_09/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


Los temas que vimos en esta parte son limpieza de datos, expresiones regulares, funciones lambda, apply .

La idea de esta práctica es ejercitar los puntos que vamos a necesitar en la clase presencial.

Usaremos el dataset de las obras del Met (The Metropolitan Museum of Art)

https://github.com/metmuseum/openaccess/



## Ejercicio 1
Vamos a leer en la variable `data` los datos del archivo /M2/CLASE_05_Limpieza_de_datos/Data/MetObjects_sample.csv en un `DataFrame` de pandas con el método `read_csv` 

Veamos de qué tipos de datos son las columnas.


In [2]:
import pandas as pd
import numpy as np
import os
%matplotlib inline

data = pd.read_csv("../Data/MetObjects_sample.csv")


## Ejercicio 2
¿Qué campos tienen valores nulos? ¿Qué porcentaje de nulos tienen cada uno de ellos?

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4743 entries, 0 to 4742
Data columns (total 52 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               4743 non-null   int64  
 1   Object Number            4743 non-null   object 
 2   Is Highlight             4743 non-null   bool   
 3   Is Public Domain         4743 non-null   bool   
 4   Is Timeline Work         4743 non-null   bool   
 5   Object ID                4743 non-null   int64  
 6   Department               4743 non-null   object 
 7   AccessionYear            3743 non-null   object 
 8   Object Name              4655 non-null   object 
 9   Title                    3914 non-null   object 
 10  Culture                  2318 non-null   object 
 11  Period                   1319 non-null   object 
 12  Dynasty                  184 non-null    object 
 13  Reign                    71 non-null     object 
 14  Portfolio               

In [4]:
nullproportion=(data.isnull().sum()/data.shape[0])*100
nullproportion

Unnamed: 0                   0.000000
Object Number                0.000000
Is Highlight                 0.000000
Is Public Domain             0.000000
Is Timeline Work             0.000000
Object ID                    0.000000
Department                   0.000000
AccessionYear               21.083702
Object Name                  1.855366
Title                       17.478389
Culture                     51.127978
Period                      72.190597
Dynasty                     96.120599
Reign                       98.503057
Portfolio                   96.963947
Artist Role                 49.736454
Artist Prefix               84.482395
Artist Display Name         49.441282
Artist Display Bio          55.471221
Artist Suffix               98.334388
Artist Alpha Sort           49.441282
Artist Nationality          66.814253
Artist Begin Date           55.787476
Artist End Date             55.977230
Artist Gender               79.084967
Artist ULAN URL             62.492094
Artist Wikid

## Ejercicio 3

Analicemos la columna Object Date

¿Qué formato tienen los valores de este campo? ¿Qué patrones pueden identificar?

Sugerencia: recuerden el método value_counts
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html

In [5]:
df=data.copy()
(df["Object Date"]).dtype
# Los valores de este campo tienen formato objeto
df["Object Date"].value_counts()

19th century              130
18th century               80
17th century               65
1912                       54
1911                       50
                         ... 
ca. 1352–1336 B.C.          1
ca. 1786–1560 B.C.          1
1917-1918                   1
664–656 B.C. or longer      1
ca. 1645                    1
Name: Object Date, Length: 1378, dtype: int64

## Ejercicio 4

Usando expresiones regulares, apply y funciones lambda, creemos un nuevo campo "ObjectDateClean" que sea de tipo numérico y tenga el valor del año corresponiente a cada registro extrayéndolo del campo ObjectDate.

¿Qué características tienen los valores de los registros que no verificaron ningún patrón de los definidos?

Ayuda: Identifiquemos los patrones y decidamos cómo extraer el año de los valores de los registros que verifican cada uno de esos patrones. Después, de a un patrón, usemos la estrategia que definimos antes para extraer el valor del año de los registros que lo verifican.

In [6]:
import re 
import numpy as np 

In [7]:
patron1_anios = re.compile(r"(\d)?\d\d\d$")

#Yo ahora quiero encontrar, que filas coinciden con mi patron1

filas_patron1 = df['Object Date'].apply(lambda x: x if x is np.NaN else re.compile(r"(\d)?\d\d\d$").search(x))
filas_patron1_notnull = filas_patron1.notnull()
#Ahora que tengo la mascara de filas con valores q condicen mi patron, puedo crear una columna nueva donde 
#guarde todos los años filtrados con mi regex, que responden a este patron

#Ahora con un apply, necesito convertir los objetos que me devuelve search, para objtener el año
df.loc[filas_patron1_notnull,'ObjectDateClean'] = filas_patron1[filas_patron1_notnull].apply(lambda x: x.group(0))



In [8]:
df.loc[filas_patron1_notnull, ["Object Date","ObjectDateClean"]].sample(10)

,Object Date,ObjectDateClean
4506,1625,1625
907,ca. 1740,1740
4233,1961,1961
3878,1890,1890
4723,ca. 1550,1550
2536,2002,2002
947,1916-1917,1917
3020,1606,1606
135,1912,1912
2758,ca. 1831,1831


Ahora quiero tomar tmb los años que vienen en formato periodo

In [9]:
patron2_anios = re.compile(r'(?P<Periodos>\d\d\d\d)\-(\d)+')

In [10]:
filas_patron2 = df['Object Date'].apply(lambda x: x if x==np.NaN else re.compile(r'(?P<Periodos>\d\d\d\d)\-(\d)+').search(str(x)))

#Ahora creo mi mascara booleana de donde tengo este patron con los años, le aplico el search por regex y lo filtro

df.loc[filas_patron2.notnull(),'ObjectDateClean']=filas_patron2[filas_patron2.notnull()].apply(lambda x: x.group('Periodos')) 

In [11]:
df.loc[filas_patron2.notnull(),['ObjectDateClean','Object Date']]

,ObjectDateClean,Object Date
133,1890,1890-1910
142,1850,ca. 1850-70
163,1916,1916-1917
253,1917,1917-1918
275,1916,1916-1917
367,1916,1916-1917
495,1916,1916-1917
551,1916,1916-1917
555,1916,1916-1917
846,1916,1916-1917


ahora voy a operar con los centurys, mismo procedimiento

In [46]:
patron3 = \
df['Object Date']\
.apply(lambda x: x if x==np.NaN else re.compile(("(?P<century>(\d)?\d)((\w\w) (C|c)entury)")).search(str(x)))

#Ahora voy a crear mi mascara booleana de los matches

mascara_booleana=patron3.notnull()

In [47]:
df.loc[mascara_booleana,'ObjectDateClean'] = \
patron3[mascara_booleana].apply(lambda x: x.group('century'))
#Ahora convierto el siglo a un periodo de año

df.loc[mascara_booleana,'ObjectDateClean'] = (df.loc[mascara_booleana,'ObjectDateClean'].astype(int)-1)*100

In [48]:
df.loc[mascara_booleana,['ObjectDateClean','Object Date']]

,ObjectDateClean,Object Date
0,1900,20th century
9,1200,11th–13th century
15,100,1st–2nd century A.D.
26,1800,19th century
27,1600,17th century
...,...,...
4728,1700,first half 18th century
4730,1200,early 13th century
4731,1900,late 19th–early 20th century
4737,200,4th–3rd century B.C.


In [49]:
(df['ObjectDateClean'].isnull().sum()/df.shape[0])*100

18.11090027408813

In [50]:
mascara_nulos=df['Object Date'].isnull()

df.loc[mascara_nulos,'ObjectDateClean']=0

In [51]:
nulos_ObjectClean=df['ObjectDateClean'].isnull()

In [52]:
df.loc[nulos_ObjectClean, ["Object Date", "ObjectDateClean"]]

,Object Date,ObjectDateClean
8,1815–30,NaN
10,ca. 1808–12,NaN
11,ca. 1295–1070 B.C.,NaN
12,1815–30,NaN
20,664–30 B.C.,NaN
...,...,...
4726,late 6th/early 5th centuries B.C.,NaN
4732,1905–6,NaN
4735,ca. 2124–1504 B.C.,NaN
4736,1825–35,NaN


Posiblemente veamos que nos faltó considerar:
* los valores terminados en BC o B.C., y marcar esos años como negativos 
* considerar los períodos que tienen siglos, en lugar de años
* borrar las C. que aparecen de sufijo
* Considerar pallabras mals escritas como "cenutry"

Si tienen ganas de seguir practicando pueden resolver algunos de éstos.
